<img src="https://drive.google.com/uc?export=view&id=1wYSMgJtARFdvTt5g7E20mE4NmwUFUuog" width="200">

[![Build Fast with AI](https://img.shields.io/badge/BuildFastWithAI-GenAI%20Bootcamp-blue?style=for-the-badge&logo=artificial-intelligence)](https://www.buildfastwithai.com/genai-course)
[![EduChain GitHub](https://img.shields.io/github/stars/satvik314/educhain?style=for-the-badge&logo=github&color=gold)](https://github.com/satvik314/educhain)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1VTMZunYgIf-VqKEYDkurtm61Hm3Ucgtx#scrollTo=yRaTWbHOc9M9)
## Master Generative AI in 6 Weeks
**What You'll Learn:**
- Build with Latest LLMs
- Create Custom AI Apps
- Learn from Industry Experts
- Join Innovation Community
Transform your AI ideas into reality through hands-on projects and expert mentorship.
[Start Your Journey](https://www.buildfastwithai.com/genai-course)
- Empowering the Next Generation of AI Innovators


##Introduction to Pydantic AI
Pydantic AI is an extension of Pydantic that allows for the creation of models that can interact with AI systems. It provides a way to define data structures with validation and serialization capabilities while integrating with AI.

###Setup
First, we need to install the required libraries. Pydantic AI is built on top of Pydantic, so we will install both.

In [38]:
!pip install -qU pydantic-ai

###Importing Libraries

In [39]:
from pydantic_ai import Agent
from pydantic import BaseModel
import os
from google.colab import userdata
import asyncio
import nest_asyncio
nest_asyncio.apply()

###Setup API Keys

In [40]:
os.environ['GEMINI_API_KEY'] = userdata.get('GEMINI_API_KEY')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

###Basic Example Using Basic Pydantic Class


This is Basic Example How to use Pydantic Class with Pydantic AI

In [41]:
#Define the Class
class CityLocation(BaseModel):
    city: str
    country: str

# Run the agent synchronously to conduct a conversation
agent = Agent('gemini-1.5-flash-8b', result_type=CityLocation)
result = agent.run_sync('Where the olympics held in 2012?')
print(result.data)
print(result.cost())

city='London' country='United Kingdom'
Cost(request_tokens=63, response_tokens=8, total_tokens=71, details=None)


###Example 2

Simple example of using PydanticAI to construct a Pydantic model from a text input.



In [42]:
import os
from pydantic import BaseModel
from pydantic_ai import Agent

# Define a Pydantic model for the expected output
class MyModel(BaseModel):
    city: str
    country: str

# Get the model name from environment variable or use a default
model_name = os.getenv('PYDANTIC_AI_MODEL', 'openai:gpt-4o')
print(f'Using model: {model_name}')

# Create the agent with the specified model and result type
agent = Agent(model_name, result_type=MyModel)


# Run the agent with a sample query
result = agent.run_sync('The windy city in the US of A.')

# Print the output data and cost
print(result.data)
print(f'Cost: {result.cost()}')

Using model: openai:gpt-4o
city='Chicago' country='USA'
Cost: Cost(request_tokens=67, response_tokens=18, total_tokens=85, details={'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0, 'cached_tokens': 0})


###Hello World with Pydantic AI Agent

Here's a minimal example of PydanticAI Agent:



In [43]:
# Define a simple agent with a static system prompt
agent = Agent(
    model='openai:gpt-4o',
    system_prompt='Be concise, reply with one sentence.'
)

# Run the agent synchronously to conduct a conversation
result = agent.run_sync('Where does "hello world" come from?')
print(result.data)

"Hello, World!" originates from the 1978 book "The C Programming Language" by Brian Kernighan and Dennis Ritchie, often used as a simple program to illustrate basic syntax in programming.


###Simple Weather Agent Code


A "Weather Agent" that streams real-time weather updates for multiple locations using latitude, longitude, and weather APIs.

* A weather API key from [tomorrow.io](https://) set via WEATHER_API_KEY
* A geocoding API key from [geocode.maps.co](https://) set via GEO_API_KEY

In [44]:
import os
import asyncio
from dataclasses import dataclass
from typing import Any, Dict

from httpx import AsyncClient
from pydantic import BaseModel
from pydantic_ai import Agent, RunContext

# Define dependencies for the agent
@dataclass
class Deps:
    client: AsyncClient
    weather_api_key: str | None
    geo_api_key: str | None

# Create the weather agent
weather_agent = Agent(
    model='openai:gpt-4o',
    system_prompt='Be concise, reply with one sentence.',
    deps_type=Deps,
    retries=2,
)

@weather_agent.tool
async def get_lat_lng(ctx: RunContext[Deps], location: str) -> Dict[str, float]:
    """Get latitude and longitude for a given location."""
    if not ctx.deps.geo_api_key:
        return {'lat': 51.1, 'lng': -0.1}  # Dummy response for testing

    params = {'q': location, 'api_key': ctx.deps.geo_api_key}
    response = await ctx.deps.client.get('https://geocode.maps.co/search', params=params)
    response.raise_for_status()

    data = response.json()
    if data:
        return {'lat': data[0]['lat'], 'lng': data[0]['lon']}
    raise ValueError('Could not find the location')

@weather_agent.tool
async def get_weather(ctx: RunContext[Deps], lat: float, lng: float) -> Dict[str, Any]:
    """Get weather information for given latitude and longitude."""
    if not ctx.deps.weather_api_key:
        return {'temperature': '21 °C', 'description': 'Sunny'}  # Dummy response for testing

    params = {
        'apikey': ctx.deps.weather_api_key,
        'location': f'{lat},{lng}',
        'units': 'metric',
    }

    response = await ctx.deps.client.get('https://api.tomorrow.io/v4/weather/realtime', params=params)
    response.raise_for_status()

    values = response.json()['data']['values']

    # Weather code mapping
    code_lookup = {
        1000: 'Clear, Sunny',
        1001: 'Cloudy',
        4001: 'Rain',
        5000: 'Snow',
        # Add more codes as needed...
    }

    return {
        'temperature': f'{values["temperatureApparent"]:0.0f}°C',
        'description': code_lookup.get(values['weatherCode'], 'Unknown'),
    }


###Example Usage

In [45]:
async def main():
    async with AsyncClient() as client:
        weather_api_key = userdata.get('WEATHER_API_KEY')
        geo_api_key = userdata.get('GEO_API_KEY')

        deps = Deps(client=client, weather_api_key=weather_api_key, geo_api_key=geo_api_key)

        result = await weather_agent.run('What is the weather like in Delhi?', deps=deps)
        print('Response:', result.data)

asyncio.run(main())

Response: The weather in Delhi is currently clear and sunny with a temperature of 25°C.


###Bank support Agent
Small but complete example of using PydanticAI to build a support agent for a bank.

In [46]:
from dataclasses import dataclass
from pydantic import BaseModel, Field
from pydantic_ai import Agent, RunContext

class FakeDatabase:
    """A mock database for demonstration purposes."""

    @classmethod
    async def get_customer_name(cls, id: int) -> str | None:
        return 'John' if id == 123 else None

    @classmethod
    async def get_customer_balance(cls, id: int) -> float:
        return 123.45 if id == 123 else None

@dataclass
class SupportDependencies:
    customer_id: int
    db: FakeDatabase

class SupportResult(BaseModel):
    support_advice: str = Field(description='Advice returned to the customer')
    block_card: bool = Field(description='Whether to block their card')
    risk: int = Field(description='Risk level of query', ge=0, le=10)

# Create the support agent
support_agent = Agent(
    model='openai:gpt-4o',
    deps_type=SupportDependencies,
    result_type=SupportResult,
    system_prompt=(
        'You are a bank support agent. Provide customer support '
        'and assess the risk level of their query. Use the customer\'s name in your response.'
    ),
)

@support_agent.system_prompt
async def add_customer_name(ctx: RunContext[SupportDependencies]) -> str:
    customer_name = await ctx.deps.db.get_customer_name(ctx.deps.customer_id)
    return f"The customer's name is {customer_name!r}"

@support_agent.tool
async def get_balance(ctx: RunContext[SupportDependencies]) -> str:
    """Returns the customer's current account balance."""
    balance = await ctx.deps.db.get_customer_balance(ctx.deps.customer_id)
    return f'${balance:.2f}'


# Example usage


In [47]:
# Example usage
deps = SupportDependencies(customer_id=123, db=FakeDatabase())
balance_result = support_agent.run_sync('What is my balance?', deps=deps)
print(balance_result.data)

loss_result = support_agent.run_sync('I just lost my card!', deps=deps)
print(loss_result.data)

support_advice='Hello John, your current account balance is $123.45. If you have any further questions, feel free to ask!' block_card=False risk=0
support_advice="John, I'm sorry to hear that you've lost your card. We're taking steps to secure your account while we assist you." block_card=True risk=8
